<a href="https://colab.research.google.com/github/anokhina-rgb/Multilingual-Corpus-for-EU-Studies/blob/main/mp3_splitter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 1️⃣ Install packages and FFmpeg
!pip install -q git+https://github.com/openai/whisper.git pydub torch torchaudio gradio
!apt-get update -qq
!apt-get install -y -qq ffmpeg

# 2️⃣ Imports
import os
import tempfile
import zipfile
from pydub import AudioSegment
import whisper
import torch
import gradio as gr

# 3️⃣ Load Whisper model
device = "cuda" if torch.cuda.is_available() else "cpu"
model_size = "base"
print("Loading Whisper model on", device)
model = whisper.load_model(model_size).to(device)
print("Model loaded.\n")

# 4️⃣ Function to process MP3s and return a ZIP
def process_mp3s_zip(filepaths, pause_seconds=10):
    pause_ms = pause_seconds * 1000
    zip_path = tempfile.NamedTemporaryFile(delete=False, suffix=".zip").name
    with zipfile.ZipFile(zip_path, "w") as zipf:
        for idx, path in enumerate(filepaths, 1):
            # Load audio
            original = AudioSegment.from_file(path)
            result = model.transcribe(path)
            silence = AudioSegment.silent(duration=pause_ms)
            final_audio = AudioSegment.empty()

            segments = result.get("segments", [])
            if not segments:
                final_audio = original
            else:
                for seg in segments:
                    start_ms = int(seg["start"] * 1000)
                    end_ms = int(seg["end"] * 1000)
                    chunk = original[start_ms:end_ms]
                    final_audio += chunk
                    final_audio += silence  # add pause after each segment

            # Save processed MP3 to temp
            tmp_out = tempfile.NamedTemporaryFile(delete=False, suffix=".mp3")
            final_audio.export(tmp_out.name, format="mp3")

            # Add processed MP3 to ZIP
            zipf.write(tmp_out.name, os.path.basename(path))

    return zip_path

# 5️⃣ Gradio interface
iface = gr.Interface(
    fn=process_mp3s_zip,
    inputs=[
        gr.File(file_types=[".mp3"], type="filepath", label="Upload MP3 files", file_count="multiple"),
        gr.Slider(1, 60, value=10, step=1, label="Pause duration (seconds)")
    ],
    outputs=gr.File(label="Download ZIP of processed MP3s"),
    title="Automated MP3 Pause Inserter",
    description="Upload MP3s, set pause duration, and download a ZIP with all processed MP3s automatically."
)

iface.launch()



  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 101.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 76.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 73.6 MB/s

100%|████████████████████████████████████████| 139M/139M [00:01<00:00, 144MiB/s]


Model loaded.

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://41e5c175a0c732252c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
